이번에는 memory 를 chain 에 꽂는 방법과 두 종류의 chain 을 사용해서 꽂는 방법을 배워보자.

첫 번째로 우리는 LLMChain 이라는 것을 사용해본다.  
LLMChain 은 off-the-shelf chain 인데 이는 일반적인 목적을 가진 chain 을 의미하고, langchain 에 아주 많고 아주 유용하다.  
하지만 우리가 스스로 무언가를 만들어 볼 때 off-the-shelf chain 보다 우리가 직접 커스텀해서 만든 chain을 활용하기를 선호할 것이다.  
이 때는 이전에 배운 것처럼 langchain expression 언어를 활용해서 우리의 것을 만들 수 있다.



### off-the-shelf chain 은 사용법

이제 우리는 우리가 가지고 있는 memory 를 어떻게 LLMChain 에 넣는지 배워보자.

In [1]:
from langchain.memory import ConversationSummaryBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate

llm = ChatOpenAI(temperature=0.1)

memory = ConversationSummaryBufferMemory(llm=llm, max_token_limit=80)

# chain 생성
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
)

# predict
chain.predict(question="My name is lution")

'Nice to meet you, Lution! How can I assist you today?'

In [18]:
chain.predict(question="I live in seoul")

"That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?"

In [19]:
chain.predict(question="what is my name?")

"I'm sorry, I do not have access to personal information such as your name."

실행해보면 잘 동작하지만 한 이것은 이전 대화를 기억하지 못한다.

자 이제 우리 chain 을 디버깅해보자.

LLMChain 에 verbose=True 값을 준다. 이 값을 chain 에게 전달하면 chain 이 실행했을 때 chain 의 프롬프트 로그들을 확인할 수 있다.

In [20]:
# chain 생성
chain = LLMChain(
    llm=llm,
    memory=memory,
    prompt=PromptTemplate.from_template("{question}"),
    verbose=True,
)

In [21]:
chain.predict(question="what is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
what is my name?

> Finished chain.


"I'm sorry, I do not have access to personal information such as your name."

In [22]:
chain.predict(question="I live in seoul")



> Entering new LLMChain chain...
Prompt after formatting:
I live in seoul

> Finished chain.


"That's great! Seoul is a vibrant and bustling city with a rich history and culture. There are so many things to see and do in Seoul, from exploring ancient palaces and temples to enjoying delicious Korean cuisine and shopping in trendy neighborhoods. What do you enjoy most about living in Seoul?"

In [23]:
chain.predict(question="what is my name?")



> Entering new LLMChain chain...
Prompt after formatting:
what is my name?

> Finished chain.


"I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information."

확인해보니 대화의 내역이 프롬프트에 계속 추가되지 않는다.

이제 우리가 할 일은 어떠한 방식으로 프롬프트에게 대화 기록을 추가해 주어야 하는 것이다.

그런데 재밌는 점은 memory 는 계속 업데이트 된다는 것이다.



In [25]:
print(memory.load_memory_variables({}))

{'history': "System: Lution introduces themselves and the AI responds by greeting Lution and asking how it can assist them. Lution mentions living in Seoul, and the AI responds by highlighting the vibrant and bustling nature of the city, prompting Lution to share what they enjoy most about living in Seoul. The human asks the AI for their name, but the AI explains it does not have access to personal information like names. The human mentions living in Seoul, and the AI expresses enthusiasm for the city's rich history and culture, prompting the human to share what they enjoy most about living in Seoul.\nHuman: what is my name?\nAI: I'm sorry, I do not know your name as I am an AI assistant and do not have access to personal information."}


response 를 보면 우리의 대화를 요약하는 건 이미 잘 동작하고 있다.

